In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import requests

In [2]:
train_data = pd.read_csv('https://github.com/HMinjae/heartfelt_project/raw/main/TF_train.txt',header = 0, delimiter = '\t', quoting=3)
train_data.head()

,id,document,label
0,1,오늘 하루도 너무 잘했어요!,1
1,2,항상 긍정적인 마음이 당신을 빛나게 해요.,1
2,3,당신의 미소가 주변을 환하게 만들어요.,1
3,4,매 순간이 소중한 추억으로 남을 거예요.,1
4,5,꿈을 향해 한 걸음 더 나아갔어요!,1


In [3]:
train_review = [review for review in train_data['document'] if type(review) is str]
train_review

['오늘 하루도 너무 잘했어요!',
 '항상 긍정적인 마음이 당신을 빛나게 해요.',
 '당신의 미소가 주변을 환하게 만들어요.',
 '매 순간이 소중한 추억으로 남을 거예요.',
 '꿈을 향해 한 걸음 더 나아갔어요!',
 '오늘도 많은 것을 배우고 성장했어요.',
 '당신의 노력이 결실을 맺을 거예요.',
 '어제보다 더 나은 오늘이에요.',
 '당신은 무엇이든 할 수 있어요.',
 '포기하지 않는 한, 성공은 시간문제예요.',
 '하루를 마무리하며 성취감을 느껴요.',
 '오늘 만난 모든 사람이 친절했어요.',
 '새로운 취미가 즐거워요.',
 '작은 성취가 큰 행복을 만들어요.',
 '건강하게 하루를 보냈어요.',
 '새로운 도전이 기대돼요.',
 '모든 것이 잘 풀려서 기분이 좋아요.',
 '소중한 사람들과 시간을 보내서 행복해요.',
 '매일 나 자신이 발전하는 것을 느껴요.',
 '오늘의 노력이 내일을 만들어요.',
 '햇볕이 따뜻해서 기분이 좋았어요.',
 '감사할 일이 많아요.',
 '의미 있게 하루를 보냈어요.',
 '문제를 해결하고 나니 마음이 가벼워요.',
 '내 삶에 감사해요.',
 '내일을 위해 잘 준비했어요.',
 '오늘 받은 칭찬이 기분을 좋게 했어요.',
 '좋은 사람들과 즐거운 시간을 보냈어요.',
 '목표를 향해 한 걸음 더 나아갔어요.',
 '오늘은 내 인생에 소중한 하루였어요.',
 '평온하고 안정된 하루였어요.',
 '맛있는 음식을 먹어서 기분이 좋아요.',
 '새로운 것을 배우는 즐거움을 느꼈어요.',
 '필요한 사람을 도와줄 수 있어서 행복해요.',
 '많은 작은 행복을 느꼈어요.',
 '오늘도 최선을 다했어요.',
 '긍정적인 생각이 많은 도움이 됐어요.',
 '내 삶이 점점 더 좋아지고 있어요.',
 '오늘은 나에게 영감을 준 하루였어요.',
 '인생 목표에 한 걸음 더 가까워진 것 같아요.',
 '새로운 친구를 만나서 행복해요.',
 '가족과 시간을 보내는 것이 행복해요.',
 '오늘은 모든 면에서 정말 완벽한 

In [4]:
print('긍정 리뷰 갯수: {}'.format(train_data['label'].value_counts()[1]))
print('부정 리뷰 갯수: {}'.format(train_data['label'].value_counts()[0]))

긍정 리뷰 갯수: 2003
부정 리뷰 갯수: 2000


In [5]:
!pip install konlpy

In [9]:
import numpy as np
import pandas as pd
import re
import json
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

train_data = pd.read_csv('https://github.com/HMinjae/heartfelt_project/raw/main/TF_train.txt', header = 0, delimiter='\t', quoting=3)

In [10]:
train_data['document'][:5]

0                     내일 할 일이 많아서 기쁘다. 가끔은 바쁜 것이 행복하다.
1    오늘 일기예보는 흐리고 비가 올 것이라고 했는데, 다행이 비는 안 왔다. 하지만 구...
2    시험 공부를 열심히 했는데, 시험에서 받은 성적이 나의 기대에는 못 미쳤다. 하지만...
3    오랜만에 친구를 만났는데, 얘기를 많이 못 나눴다. 다음에는 좀 더 시간을 가지고 ...
4    오늘 운동을 했는데, 제대로 된 운동량을 채우지 못했다. 하지만 조금이라도 한 것이...
Name: document, dtype: object

In [11]:
#전처리 함수 만들기
def preprocessing(review, okt, remove_stopwords = False, stop_words =[]):
  #함수인자설명
  # review: 전처리할 텍스트
  # okt: okt객체를 반복적으로 생성하지 않고 미리 생성 후 인자로 받음
  # remove_stopword: 불용어를 제거할지 여부 선택. 기본값 False
  # stop_words: 불용어 사전은 사용자가 직접 입력, 기본값 빈 리스트

  # 1. 한글 및 공백 제외한 문자 모두 제거
  review_text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]','',review)

  #2. okt 객체를 활용하여 형태소 단어로 나눔
  word_review = okt.morphs(review_text,stem=True)

  if remove_stopwords:
    #3. 불용어 제거(선택)
    word_review = [token for token in word_review if not token in stop_words]
  return word_review

In [12]:
# 전체 텍스트 전처리
stop_words = ['은','는','이','가','하','아','것','들','의','있','되','수','보','주','등','한']
okt = Okt()
clean_train_review = []

for review in train_data['document']:
  # 리뷰가 문자열인 경우만 전처리 진행
  if type(review) == str:
    clean_train_review.append(preprocessing(review,okt,remove_stopwords=True,stop_words= stop_words))
  else:
    clean_train_review.append([]) #str이 아닌 행은 빈칸으로 놔두기

clean_train_review[:4]

[['내일', '하다', '일이', '많다', '기쁘다', '가끔', '바쁘다', '행복하다'],
 ['오늘',
  '일기예보',
  '흐리다',
  '비',
  '오다',
  '이라고',
  '하다',
  '다행',
  '비다',
  '안',
  '오다',
  '하지만',
  '구름',
  '많다',
  '그다지',
  '맑다',
  '않다'],
 ['시험',
  '공부',
  '를',
  '열심히',
  '하다',
  '시험',
  '에서',
  '받다',
  '성적',
  '나',
  '기대',
  '에는',
  '못',
  '미치다',
  '하지만',
  '떨어지다',
  '않다',
  '다행',
  '이라',
  '생각',
  '하다'],
 ['오랜',
  '만',
  '에',
  '친구',
  '를',
  '만나다',
  '얘기',
  '를',
  '많이',
  '못',
  '나누다',
  '다음',
  '에는',
  '좀',
  '더',
  '시간',
  '을',
  '가지',
  '고',
  '재미있다',
  '이야기',
  '를',
  '나누다']]

In [13]:
test_data = pd.read_csv('https://github.com/HMinjae/heartfelt_project/raw/main/TF_train.txt', header = 0, delimiter='\t', quoting=3)

clean_test_review = []
for review in test_data['document']:
  if type(review) == str:
    clean_test_review.append(preprocessing(review, okt, remove_stopwords=True, stop_words=stop_words))
  else:
    clean_test_review.append([])

In [14]:
# 인덱스 벡터 변환 후 일정 길이 넘어가거나 모자라는 리뷰 패딩처리
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_train_review)
train_sequences = tokenizer.texts_to_sequences(clean_train_review)
test_sequences = tokenizer.texts_to_sequences(clean_test_review)

word_vocab = tokenizer.word_index #단어사전형태
MAX_SEQUENCE_LENGTH = 1000 #문장 최대 길이

#학습 데이터
train_inputs = pad_sequences(train_sequences,maxlen=MAX_SEQUENCE_LENGTH,padding='post')

#학습 데이터 라벨 벡터화
train_labels = np.array(train_data['label'])

#평가 데이터
test_inputs = pad_sequences(test_sequences,maxlen=MAX_SEQUENCE_LENGTH,padding='post')
#평가 데이터 라벨 벡터화
test_labels = np.array(test_data['label'])

In [15]:
# 학습 데이터 불러오기
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
from tqdm import tqdm

In [16]:
import numpy as np
import requests
from io import BytesIO
from tensorflow.keras.preprocessing.sequence import pad_sequences

# nsmc_train_input.npy 파일 다운로드
response = requests.get('https://github.com/HMinjae/heartfelt_project/raw/main/DATA/CLEAN_DATA/nsmc_train_input.npy')
train_input = np.load(BytesIO(response.content))
train_input = pad_sequences(train_input, maxlen=train_input.shape[1])

# nsmc_train_label.npy 파일 다운로드
response = requests.get('https://github.com/HMinjae/heartfelt_project/raw/main/DATA/CLEAN_DATA/nsmc_train_label.npy')
train_label = np.load(BytesIO(response.content))

# `data_configs.json` 파일 다운로드
response = requests.get('https://github.com/HMinjae/heartfelt_project/raw/main/DATA/CLEAN_DATA/data_configs.json')
prepro_configs = json.loads(response.text)

In [17]:
model_name= 'cnn_classifier_kr'
BATCH_SIZE = 512
NUM_EPOCHS = 10
VALID_SPLIT = 0.1
MAX_LEN = train_input.shape[1]

kargs={'model_name': model_name, 'vocab_size':prepro_configs['vocab_size'],'embbeding_size':128, 'num_filters':100,'dropout_rate':0.5, 'hidden_dimension':250,'output_dimension':1}

In [18]:
class CNNClassifier(tf.keras.Model):

  def __init__(self, **kargs):
    super(CNNClassifier, self).__init__(name=kargs['model_name'])
    self.embedding = layers.Embedding(input_dim=kargs['vocab_size'], output_dim=kargs['embbeding_size'])
    self.conv_list = [layers.Conv1D(filters=kargs['num_filters'], kernel_size=kernel_size, padding='valid',activation = tf.keras.activations.relu,
                                    kernel_constraint = tf.keras.constraints.MaxNorm(max_value=3)) for kernel_size in [3,4,5]]
    self.pooling = layers.GlobalMaxPooling1D()
    self.dropout = layers.Dropout(kargs['dropout_rate'])
    self.fc1 = layers.Dense(units=kargs['hidden_dimension'],
                            activation = tf.keras.activations.relu,
                            kernel_constraint=tf.keras.constraints.MaxNorm(max_value=3.))
    self.fc2 = layers.Dense(units=kargs['output_dimension'],
                            activation=tf.keras.activations.sigmoid,
                            kernel_constraint= tf.keras.constraints.MaxNorm(max_value=3.))


  def call(self,x):
    x = self.embedding(x)
    x = self.dropout(x)
    x = tf.concat([self.pooling(conv(x)) for conv in self.conv_list], axis = 1)
    x = self.fc1(x)
    x = self.fc2(x)
    return x


In [19]:
from tensorflow.keras.models import save_model

In [20]:
model = CNNClassifier(**kargs)
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss = tf.keras.losses.BinaryCrossentropy(),
              metrics = [tf.keras.metrics.BinaryAccuracy(name='accuracy')])

In [21]:
INPUT_TEST_DATA = 'nsmc_test_input.npy'
LABEL_TEST_DATA = 'nsmc_test_label.npy'
SAVE_FILE_NM = 'model.keras'


# nsmc_train_input.npy 파일 다운로드
response = requests.get('https://github.com/HMinjae/heartfelt_project/raw/main/DATA/CLEAN_DATA/nsmc_test_input.npy')
test_input = np.load(BytesIO(response.content))
test_input = pad_sequences(test_input, maxlen=test_input.shape[1])

# nsmc_train_label.npy 파일 다운로드
response = requests.get('https://github.com/HMinjae/heartfelt_project/raw/main/DATA/CLEAN_DATA/nsmc_test_label.npy')
test_label_data = np.load(BytesIO(response.content))


In [22]:
import tensorflow as tf
import requests

# 모델 파일 다운로드
url = 'https://github.com/HMinjae/heartfelt_project/raw/main/DATA/my_models/model.keras'
r = requests.get(url)
open('model.keras', 'wb').write(r.content)

# 모델 로드
model = model = CNNClassifier(**kargs)

# 모델 컴파일 (필요시)
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy')])

# 평가 실행
model.evaluate(test_input, test_label_data)


63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2585 - loss: 0.6994


[0.6940562129020691, 0.47626185417175293]

In [24]:
import numpy as np
import pandas as pd
import re
import json
import requests
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import load_model
okt = Okt()
tokenizer  = Tokenizer()


urla = 'https://raw.githubusercontent.com/HMinjae/heartfelt_project/main/DATA/CLEAN_DATA/data_configs.json'

# URL에서 데이터를 가져옵니다.
response = requests.get(urla)

# 응답을 JSON으로 파싱합니다.
prepro_configs = json.loads(response.text)
prepro_configs['vocab'] = word_vocab

tokenizer.fit_on_texts(word_vocab)

MAX_LENGTH = 1000 #문장최대길이

sentence = input('일기를 입력해주세요.: ')
#print(sentence)
sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣\\s ]','', sentence)
stopwords = ['은','는','이','가','하','아','것','들','의','있','되','수','보','주','등','한'] # 불용어 추가할 것이 있으면 이곳에 추가
sentence = okt.morphs(sentence, stem=True) # 토큰화
sentence = [word for word in sentence if not word in stopwords] # 불용어 제거
vector  = tokenizer.texts_to_sequences(sentence)
pad_new = pad_sequences(vector, maxlen = MAX_LENGTH) # 패딩
#print('변환된 문자 결과값: ',pad_new)
model = model = CNNClassifier(**kargs)
predictions = model.predict(pad_new)
predictions = float(predictions.squeeze(-1)[1])
#print(predictions)
if(predictions > 0.75):
  print("{:.2f}% 확률로 기분 매우 좋음입니다.\n".format(predictions * 100))
elif(predictions > 0.5):
  print("{:.2f}% 확률로 기분 좋음입니다.\n".format(predictions * 100))
elif(predictions > 0.25):
  print("{:.2f}% 확률로 기분 안 좋음입니다.\n".format(predictions * 100))
else:
  print("{:.2f}% 확률로 기분 매우 안 좋음입니다.\n".format((predictions) * 100))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
50.56% 확률로 기분 좋음입니다.

